In [1]:
import os
import sys
sys.path.append('../../')

import json
import yaml
import pandas as pd
import numpy as np

from src.utils import get_tweet_filepaths

In [20]:
main_dir = '../..'
config_path = os.path.join(main_dir, 'config/config.json')

#load config file
assert os.path.exists(config_path)
with open(config_path) as file:
    config = json.load(file)

In [3]:
tweet_filepaths = get_tweet_filepaths(main_dir, config)
tweet_filepaths

['../../data/tweets/tweets_auth7_ckpt17.json',
 '../../data/tweets/tweets_auth0_ckpt24.json',
 '../../data/tweets/tweets_auth6_ckpt18.json',
 '../../data/tweets/tweets_auth2_ckpt11.json',
 '../../data/tweets/tweets_auth7_ckpt4.json',
 '../../data/tweets/tweets_auth0_ckpt0.json',
 '../../data/tweets/tweets_auth4_ckpt10.json',
 '../../data/tweets/tweets_auth6_ckpt20.json',
 '../../data/tweets/tweets_auth6_ckpt14.json',
 '../../data/tweets/tweets_auth3_ckpt26.json',
 '../../data/tweets/tweets_auth3_ckpt21.json',
 '../../data/tweets/tweets_auth5_ckpt4.json',
 '../../data/tweets/tweets_auth6_ckpt9.json',
 '../../data/tweets/tweets_auth3_ckpt10.json',
 '../../data/tweets/tweets_auth7_ckpt12.json',
 '../../data/tweets/tweets_auth6_ckpt15.json',
 '../../data/tweets/tweets_auth0_ckpt5.json',
 '../../data/tweets/tweets_auth5_ckpt1.json',
 '../../data/tweets/tweets_auth3_ckpt18.json',
 '../../data/tweets/tweets_auth1_ckpt28.json',
 '../../data/tweets/tweets_auth2_ckpt12.json',
 '../../data/tweets

In [4]:
#hashtag count dictionary
hashtag_dict = {}

#specific hashtag tweet dictionaries
qanon = {}
hnerds = {}
fakenews = {}
for filepath in tweet_filepaths:
    
    try:
        #files are large so loading each file can take a while
        with open(filepath) as json_file:
            print(f'Loading {filepath}')
            tmp_tweets = json.load(json_file)

        #for each user's tweets: count hashtags and extract tweets containing specific hashtags
        for user in tmp_tweets:
            curr_user_tweets = tmp_tweets[user]
            
            for tweet_id in curr_user_tweets:
                tweet = curr_user_tweets[tweet_id]
                
                hashtags = tweet['entities']['hashtags']
                for tag in hashtags:

                    #collect tweets containing specific hashtags
                    if tag['text'].lower() == 'qanon' or tag['text'].lower() == 'wwg1wga':
                        qanon[tweet_id] = tweet
                    if tag['text'].lower() == '12hnerds':
                        hnerds[tweet_id] = tweet
                    if tag['text'].lower() == 'fakenews':
                        fakenews[tweet_id] = tweet

                    #increment count of all occuring hashtags
                    if tag['text'].lower() in hashtag_dict:
                        count = hashtag_dict[tag['text'].lower()]
                        hashtag_dict[tag['text'].lower()] = count + 1
                    else:
                        hashtag_dict[tag['text'].lower()] = 1
        
        del tmp_tweets
        
    except Exception as e:
        print(e)
        print(f"Skipping file {filepath}")

Loading ../../data/tweets/tweets_auth7_ckpt17.json
AhmedMousa2018
WolDenis
Aib1960
h2ppla
Bikofaust
FiggeLukas
BastroBa
njeselpriem
andersteacher
rei_chri
Krimskrams5
Bredrax
conspyraciboi
beatebiber
RojosTabea
alido_olivari
GabellaLos
senicegmxde1
philipp_sndkstn
HSchubrakete
lpmitchellrtr
versagergeblie1
stefan_wittner
RalfSch98827130
lidstroem1
Schnurtzelchen2
KatJA_Opatija
ufrechhhh
Goldie41667419
roklu2
JohnBec73591932
Ribolana
DirkausD
Lalala10699115
Alexand82562097
ProPatriaA
Chriso84250618
AfDRatsgruppe
EberhardMost21
BadischZieserli
andreasguenth22
florian_ku
borfragmxch
KummererSabrina
Deniz_Sanli35
toni_sad2
AfDOrtenau
TWaskowsky
kassenpatient1
BeyEllen
JojoBach385
TheSinMin
kanzlerin3
Real_Tlacatl
HenkerKapitan
Alex07246552
dlnshin12
Womo20001
markus6822
Coyote60457520
JoeFred111
Reinhard421
PitSchneiders
OlePetzold
Volkverdraehter
GretaThunfisch
_Dennis_L_
Nh13265448
denexbola4
Thomas81244249
EssentiaSociety
DrSpassbart
getthenetcat70
chanchumucgu
maxdigitalbuch
AlissaWail

Loading ../../data/tweets/tweets_auth2_ckpt11.json


KeyboardInterrupt: 

In [13]:
tweets_path = os.path.join(main_dir, config['tweets_dir'])

with open(os.path.join(tweets_path, 'qanon_tweets.json'),'w') as json_file:
    json.dump(qanon, json_file)  
with open(os.path.join(tweets_path, 'hnerds_tweets.json'),'w') as json_file:
    json.dump(hnerds, json_file)  
with open(os.path.join(tweets_path, 'fakenews_tweets.json'),'w') as json_file:
    json.dump(fakenews, json_file)  

In [22]:
#save all hashtag counts as csv
df_hashtags = pd.DataFrame()
df_hashtags['tag'] = hashtag_dict.keys()
df_hashtags['count'] =  hashtag_dict.values()

results_path= os.path.join(main_dir, config['results_dir'])
df_hashtags.to_csv(os.path.join(results_path, 'hashtags_all.csv'), index=False)